In [ ]:
import sys, os
# path to SpaOTsc
sys.path.append('/home/SpaOTsc')
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from spaotsc import SpaOTsc
from sklearn.metrics import matthews_corrcoef, average_precision_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.preprocessing import binarize, normalize
from sklearn.mixture import GaussianMixture
from scipy import sparse
import seaborn as sns

%pwd

In [ ]:
def preprocess_pathway(root_dir):
    file_name = 'noise_cc_interactions'
    lrpair_name = 'lrpair.csv'
    pathway_name = 'pathway.csv'

    data_large = pd.read_csv(os.path.join(root_dir, 'cci.csv'))
    grn_data = pd.read_csv(os.path.join(root_dir, 'grn.csv'))
    # * L-R pairs
    filtered = data_large[['ligand', 'receptor']]
    filtered['species'] = 'Human'
    filtered.index = range(1, filtered.shape[0] + 1)
    filtered.drop_duplicates().to_csv(os.path.join(root_dir, lrpair_name))
    print('finish lrpair', os.path.join(root_dir, lrpair_name))

    # * pathway
    pathway = grn_data[['regulator', 'target']]
    pathway = pathway.rename(columns={'regulator': 'src', 'target': 'dest'})
    pathway['pathway'] = 'Various types of N-glycan biosynthesis'
    pathway['source'] = 'KEGG'
    pathway['type'] = 'Process(missing)'
    pathway['src_tf'] = 'YES'
    pathway['dest_tf'] = 'YES'
    pathway.loc[pathway['dest'].str.slice(4).astype(int) >= 54, 'dest_tf'] = 'NO'
    pathway['species'] = 'Human'
    pathway.index = range(1, pathway.shape[0] + 1)
    pathway.to_csv(os.path.join(root_dir, pathway_name))
    print('finish pathway', os.path.join(root_dir, pathway_name))
    
def self_pcc_mat(x, progress=False):
    x_minus_mu = np.empty_like(x)
    for i in range(x.shape[0]):
        x_minus_mu[i,:] = x[i,:] - np.mean(x[i,:])
    x_std = np.empty([x.shape[0]], float)
    for i in range(x.shape[0]):
        x_std[i] = np.linalg.norm(x_minus_mu[i,:])
        if x_std[i] == 0: x_std[i] = 1
    pmat = np.ones( [x.shape[0], x.shape[0]], float )
    for i in range(x.shape[0]-1):
        for j in range(i+1, x.shape[0]):
            c = np.dot(x_minus_mu[i,:], x_minus_mu[j,:]) / (x_std[i]*x_std[j])
            pmat[i,j] = c; pmat[j,i] = c
    return pmat

def get_mcc(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    TN = np.sum(np.logical_and(pred_labels == 0, true_labels == 0))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))
    FN = np.sum(np.logical_and(pred_labels == 0, true_labels == 1))
    mcc = (TP * TN) - (FP * FN)
    denom = np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    if denom==0:
        return 0
    return mcc / denom

def preprocess_2(root_dir):
    file_name = 'bcmk_data_position_expression_real_cells.csv'
    sc_file_name = 'bcmk_sc_data_processed.txt'
    bin_file_name = 'sc_isg_binarized.txt'
    mcc_file_name = 'sc_is_mcc.npy'
    pcc_file = 'pcc_pca20.npy'
    df_sc_large = pd.read_csv(os.path.join(root_dir, 'counts.csv'), index_col=0).T
    genes = df_sc_large.columns.values

    # normalize
    df = df_sc_large
    x = np.array( df.values, float )
    x = np.log( x + 1.0 )
    for i in range(x.shape[1]):
        if np.std(x[:,i]) == 0:
            x[:, i] = 0.001
        else:
            x[:,i] = (x[:,i] - np.mean(x[:,i]))/np.std(x[:,i])
    df_new = pd.DataFrame(data=x, columns=df.columns.values, index=df.index.values)
    df_new.to_csv(os.path.join(root_dir, sc_file_name), sep='\t')
    df_sc_large.T.to_csv(os.path.join(root_dir, "giotto_counts.txt"), sep='\t')
    
    pcc_mat = self_pcc_mat(x, progress=True)
    np.save(os.path.join(root_dir, pcc_file), pcc_mat)
    
    df_sc = pd.read_csv(os.path.join(root_dir, sc_file_name), sep='\t', index_col=0)
    genes = df_sc.columns.values
    sp_genes = genes
    sp_genes = list(sp_genes)
    x_sc = np.array( df_sc.values, float )
    sc_genes = list( df_sc.columns.values )
    gind = []
    for g in sp_genes:
        gind.append(sc_genes.index(g))
    gind = np.array(gind, int)
    x_sc = x_sc[:, gind]
    df_sc_part = pd.DataFrame(data=x_sc, columns=sp_genes)
    x_sc_bin = np.empty_like( x_sc )
    for i in range(x_sc.shape[1]):
        GM = GaussianMixture(n_components=2)
        tmp = GM.fit_predict(x_sc[:,i].reshape(-1,1))
        if GM.means_[0,0] > GM.means_[1,0]:
            x_sc_bin[:,i] = 1.0 - tmp[:]
        else:
            x_sc_bin[:,i] = tmp[:]
    df_sc_bin = pd.DataFrame(data=x_sc_bin, columns=sp_genes)
    df_sc_bin.to_csv(os.path.join(root_dir, bin_file_name), sep='\t')
    
    loc_data = pd.read_csv(os.path.join(root_dir, 'locs.csv'), index_col=0)
    loc_data.to_csv(os.path.join(root_dir, 'spatalk_sc_st_data.csv'))
    loc_data.to_csv(os.path.join(root_dir, 'giotto_loc.csv'), header=False, index=False)
    
    print('finish binarized', os.path.join(root_dir, bin_file_name))
    sc_st_data = pd.read_csv(os.path.join(root_dir, 'spatalk_sc_st_data.csv'))
    is_dmat_large =squareform(pdist(sc_st_data.loc[:, ['x', 'y']],metric='euclidean'))
    print(is_dmat_large.shape)
    sparse.save_npz(os.path.join(root_dir, 'is_mat.npz'), sparse.coo_matrix(is_dmat_large))
    
    mcc_all = np.asarray([get_mcc(p, g) for p in x_sc_bin for g in x_sc_bin])
    mcc_all = mcc_all.reshape(len(x_sc_bin),-1)
    np.save(os.path.join(root_dir, mcc_file_name), mcc_all)
    

def run(root_dir):
    sc_st_data = pd.read_csv(os.path.join(root_dir, 'spatalk_sc_st_data.csv'))
    is_dmat_large =squareform(pdist(sc_st_data.loc[:, ['x', 'y']],metric='euclidean'))
    sparse.save_npz(os.path.join(root_dir, 'is_mat.npz'), sparse.coo_matrix(is_dmat_large))
    df_sc = pd.read_csv(os.path.join(root_dir, 'bcmk_sc_data_processed.txt') , sep='\t', index_col=0)
    df_sc_bin = pd.read_csv(os.path.join(root_dir, 'sc_isg_binarized.txt'), sep='\t', index_col=0)

    df_is_bin = df_sc_bin
    sc_pcc = np.load(os.path.join(root_dir, 'pcc_pca20.npy'))
    mcc = np.load(os.path.join(root_dir, 'sc_is_mcc.npy'))
    issc = SpaOTsc.spatial_sc(
        sc_data=df_sc,
        sc_data_bin=df_sc_bin,
        is_data_bin=df_is_bin,
        sc_dmat = np.exp(1-sc_pcc),
        is_dmat=is_dmat_large)

    issc.cell_cell_distance(sc_dmat_spatial=is_dmat_large)
    issc.clustering(pca_n_components=2)
    issc.nonspatial_correlation()
    return issc
    
    
def get_result(root_dir, issc):
    lr_pair = pd.read_csv(os.path.join(root_dir, 'lrpair.csv')).loc[:, ['ligand', 'receptor']]
    cell_info = pd.read_csv(os.path.join(root_dir, 'cell_info.csv'), index_col=0)
    ct_num = len(np.unique(cell_info.loc[:, "cell.type.idx"].to_numpy()))
    idx_dict = {}
    for i in range(1, ct_num+1):
        idx_dict[i] = cell_info['cell.type.idx'] == i
    S_name_a_b = {}
    for l, r in lr_pair.values.astype(str):
        ccc = issc.spatial_signaling_ot([l], [r])
        S_cluster_a_b = np.zeros([ct_num, ct_num], float)
        for clusterA in range(1, ct_num+1):
            for clusterB in range(1, ct_num+1):
                idx_a = idx_dict[clusterA]
                idx_b = idx_dict[clusterB]
                C = ccc[idx_a,:][:,idx_b]
                # take average of all cells from this cell-type pair in the score matrix from SpaOTsc
                S_cluster_a_b[clusterA-1, clusterB-1] = np.mean(C)
        S_name_a_b[f"{l}-{r}"] = S_cluster_a_b
    col_name = ['CT'+str(i)+'-'+'CT'+str(j) for i in range(1, 6) for j in range(1, 6)]
    spaot_rlt = pd.DataFrame(index=col_name)
    for pre_pair in S_name_a_b.keys():
        spaot_rlt[pre_pair] = S_name_a_b[pre_pair].reshape(-1)
    spaot_rlt.T.to_csv(os.path.join(root_dir, 'spaot_results.csv'))

In [ ]:
dirs = []

for tree in [1, 3, 5]:
    for ncell in [500, 800]:
        for ngene in [110, 200, 500]:
            for sigma in [0.1, 0.5]:
                for seed in [1, 2, 3, 4]:
                    root_dir = f"~/scMultiSim/bench/unif/0/tree{tree}_{ncell}_cells{ngene}_genes_sigma{sigma}_{seed}/cci/"
                    dirs.append(root_dir)

In [ ]:
for root_dir in dirs:
    preprocess_pathway(root_dir)
    preprocess_2(root_dir)
    issc = run(root_dir)
    get_result(root_dir, issc)